In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
diabetes = pd.read_csv('C:\\Users\\admin2\\Documents\\Python\\DataScience_New\\DataFrame\\sber\\diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [23]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [19]:
#Создадим маску дубликатов с помощью метода duplicated() и произведём фильтрацию. 
# Результат заносим в переменную sber_duplicates. Выведем число строк в результирующем DataFrame:
dupl_col = list(diabetes.columns)
dupl_col.remove('Pregnancies')

mask = diabetes.duplicated(subset=dupl_col)
diabetes_dupl = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_dupl.shape[0]}')

Число найденных дубликатов: 10


In [20]:
diabetes_df = diabetes.drop_duplicates(subset=dupl_col)
print(f'Результирующее число записей: {diabetes_df.shape[0]}')

Результирующее число записей: 768


In [21]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [25]:
#мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop()
information_diabetes = diabetes_df.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')

Результирующее число признаков: 9


In [26]:
information_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 60.0 KB


In [34]:
diabetes_list=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in diabetes_list:
    information_diabetes[col]=information_diabetes[col].apply(lambda x: np.nan if x==0 else x)
    
null_percent = information_diabetes.isnull().mean()
cols_null = null_percent[null_percent>0].sort_values(ascending=False)
display(round(null_percent['Insulin'],2))

0.49

In [39]:
null_data = information_diabetes.isnull().sum() #вычисляем суммарное количество пропусков в столбцах
cols = null_data[null_data > 0].index #получаем имена столбцов, где число пропусков >0
fig = px.imshow(    
    information_diabetes[cols].isnull().astype('int'),
    title='Heatmap of NaN',
    width=500,
    height=300
)
fig.show()

In [46]:
diabetes_df2 = information_diabetes.copy()
thresh = diabetes_df2.shape[0]*0.7
diabetes_df2 = diabetes_df2.dropna(how='any', thresh=thresh, axis=1)
#diabetes_df2 = diabetes_df2.dropna(how='any', axis=0)
print('Результирующее число пропусков в таблице:', diabetes_df2.isnull().mean().sum())
print('Результирующий размер таблицы:', diabetes_df2.shape)
print('Удаленные столбцы:', set(information_diabetes.columns) - set(diabetes_df2.columns))
print('Число удаленных строк: ', information_diabetes.shape[0] - diabetes_df2.shape[0])

Результирующее число пропусков в таблице: 0.36197916666666674
Результирующий размер таблицы: (768, 8)
Удаленные столбцы: {'Insulin'}
Число удаленных строк:  0


In [42]:
print(f'Результирующее число признаков: {diabetes_df2.shape[1]}')

Результирующее число признаков: 8


In [47]:
#удление строк с пропуском более 2-х
m = diabetes_df2.shape[1]
diabetes_df2 = diabetes_df2.dropna(thresh=m-2, axis=0)
print(diabetes_df2.shape[0])

761


In [48]:
diabetes_df2.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'BMI',
       'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [52]:
diabetes_df2 = diabetes_df2.fillna({
    'SkinThickness': diabetes_df2['SkinThickness'].median()
    
})
print('Среднее значение:', diabetes_df2.mean())

Среднее значение: Pregnancies                   3.840999
Glucose                     121.932540
BloodPressure                72.405184
SkinThickness                29.109067
BMI                          32.457464
DiabetesPedigreeFunction      0.474126
Age                          33.323259
Outcome                       0.350854
dtype: float64


In [60]:
#Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness
def outliers_iqr_mod(diabetes_df, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(diabetes_df2[feature])
    else:
        x= diabetes_df2[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = diabetes_df2[(x<lower_bound) | (x > upper_bound)]
    cleaned = diabetes_df2[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes_df2, 'DiabetesPedigreeFunction')
print(outliers.shape[0])

29


In [62]:
def outliers_z_score(diabetes_df2, feature, log_scale=False):
    if log_scale:
        x = np.log(diabetes_df2[feature]+1)
    else:
        x = diabetes_df2[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = diabetes_df2[(x < lower_bound) | (x > upper_bound)]
    cleaned = diabetes_df2[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score(diabetes_df2, 'DiabetesPedigreeFunction')
print(outliers.shape[0])

11
